## Final Project Submission

Please fill out:
* Student name: Solphine Joseph
* Student pace:Hybrid
* Scheduled project review date/time: 
* Instructor name: Nikita/Diana
* Blog post URL:


                     
## BUSINESS UNDERSTANDING 
INTRODUCTION

Microsoft company is a leading company in technology that develops a wide range of software,services and devices.However the company has not yet ventured into movie production .This project aims at coming up with analyis of various factors that will provide the stakeholders of the company with real_time based analysis of what to consider before launching into movie production space.Some of the factors to be analyzed include:

- Studios that are doing well in the movie industry

- The top genres that are leading in the leading movie  studios

- Analyze the relationship between the total gross and the leading genres 

- Perform an analysis that identifies the top performing directors 



OBJECTIVES 

- To help microsoft leverage the data analysis carried out in order to understand the preferences of their target audience .

- To identify the best directors in the space 

- To ensure the company is making profit at the end of the day


           

####

## DATA UNDERSTANDING 
Data required in this project is sourced from :

1 https://www.imdb.com/
 
2 https://www.boxofficemojo.com/date/?ref_=bo_nb_shs_tab

3 https://www.rottentomatoes.com/

4 https://www.themoviedb.org/
          
5 https://www.the-numbers.com/'




The data has been compiled from the internet and compressed to a folder that gives access to all required information.This data was preffered since it is up to date.

## METHODOLOGY

The methodology seeks to show the data cleaning and analysis procedures in this project.


In [2]:
#Import relevant libraries and connect to SQlite Database
import pandas as pd
import csv
import sqlite3
import matplotlib.pyplot as plt



1. READING THE DATA

In [33]:
#Read  movie_gross data and put it in a dataframe defined fin-data
gross_data=pd.read_csv('zippedData/bom.movie_gross.csv.gz')


In [23]:
#Read and extract imdb database into destination directory
import zipfile
# Specify the path to the zip file
zip_file_path = 'zippedData\im.db.zip'
# Open the zip file for reading
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#Extract the files to a directory
  zip_ref.extractall('destination_directory') 


In [25]:
#connect to the Database in destination_directory

# Specify the path to the SQLite database file
db_file_path = 'destination_directory/im.db'

# Connect destination_directoryto the SQLite database
connection = sqlite3.connect(db_file_path)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Now you can execute SQL queries using the cursor
# For example, you can fetch data from a table
##cursor.execute('SELECT * FROM your_table_name')

# Fetch the results of the query
#rows = cursor.fetchall()

# Print the results
#for row in rows:
   # print(row)

# Don't forget to close the cursor and the connection when you're done
#cursor.close()
#connection.close()


     1.1 Successful studios in relation to the gross proceeds

This process entails cleaning the movie gross data that is crucial in understanding which companies are doing well in relation to the their gross  income and conduct some analysis as well


        1.1.1 Cleaning the Data

In [34]:
# Display data from BOM that shows the bom.movie_gross proceeds 
gross_data

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [59]:
# Clean the data :Replace the null values in foreign gross and domestic_grosss columns
# Check 'foreign_gross'column  Before Modification
before_foreign_nulls = gross_data['foreign_gross'].isnull().sum()
before_domestic_nulls = gross_data['domestic_gross'].isnull().sum()

#Replace the null values
gross_data['foreign_gross'].fillna(0, inplace=True)
gross_data['domestic_gross'].fillna(0, inplace=True)

# Check 'foreign_gross'column  After Modification
after_foreign_nulls = gross_data['foreign_gross'].isnull().sum()
after_domestic_nulls = gross_data['domestic_gross'].isnull().sum()

#Assert the code above is correct
assert before_foreign_nulls == 0, "There were null values in 'foreign_gross' column before modification."
assert before_domestic_nulls == 0, "There were null values in 'domestic_gross' column before modification."
assert after_foreign_nulls == 0, "The null values in 'foreign_gross' column were not replaced with zeros."
assert after_domestic_nulls == 0, "The null values in 'domestic_gross' column were not replaced with zeros."

print("All assertions passed. Null values in 'foreign_gross' and 'domestic_gross' columns were replaced with zeros.")
gross_data

All assertions passed. Null values in 'foreign_gross' and 'domestic_gross' columns were replaced with zeros.


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,Inception,WB,292600000.0,535700000.0,2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,0.0,2018
3383,Edward II (2018 re-release),FM,4800.0,0.0,2018
3384,El Pacto,Sony,2500.0,0.0,2018
3385,The Swan,Synergetic,2400.0,0.0,2018


In [41]:
#show the datatypes in the different columns
print(gross_data.dtypes)

title              object
studio             object
domestic_gross    float64
foreign_gross      object
year                int64
dtype: object


In [56]:
#Change the foreign_gross datatype to float 
# Check the current data type of 'foreign_gross' column before modification
before_dtype = gross_data['foreign_gross'].dtype

# Run the Code
gross_data.replace(',','', regex=True, inplace=True)
gross_data['foreign_gross']=gross_data['foreign_gross'].astype(float)

#check the data type of 'foreign_gross' column after modification
after_dtype = gross_data['foreign_gross'].dtype
after_dtype

#assert the changes made
assert after_dtype == float, "The data type of 'foreign_gross' column was not changed to float."



1.1.2 Exploratory Data Analysis of BOM Data


In [87]:
#Add a column to show total gross income
gross_data['total_gross']=gross_data['domestic_gross']+gross_data['foreign_gross']
gross_data['total_gross']=gross_data['total_gross'].astype('int64')
